In [1]:
from google.cloud import bigquery
bq = bigquery.Client()



q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [2]:
%load_ext google.cloud.bigquery



In [3]:
#Q1

q1_query = bq.query(
"""
SELECT ROUND((SUM(area_water_meters) / (SUM(area_land_meters) + SUM(area_water_meters))) * 100, 2) AS water_percentage
FROM 
  bigquery-public-data.geo_us_boundaries.counties
WHERE 
  county_name = 'Dane' AND state_fips_code = '55'
""")

float(sum(q1_query.to_dataframe()["water_percentage"]))

3.31

In [4]:
#Q2


q2_query = bq.query(
"""
SELECT 
  s.state,
  COUNT(c.county_fips_code) AS num_counties
FROM 
  bigquery-public-data.geo_us_boundaries.counties c
JOIN 
  bigquery-public-data.geo_us_boundaries.states s
ON 
  c.state_fips_code = s.state_fips_code
GROUP BY 
  s.state
ORDER BY 
  num_counties DESC
LIMIT 5
""")

df = q2_query.to_dataframe()

result_dict = df.set_index('state')['num_counties'].to_dict()

result_dict


{'TX': 254, 'GA': 159, 'VA': 133, 'KY': 120, 'MO': 115}

In [5]:
#Q3
from google.cloud import bigquery
client = bigquery.Client()

# Define a function to calculate cost from bytes billed
def calculate_cost(bytes_billed, price_per_tb):
    return (bytes_billed / (1024**4)) * price_per_tb  

current_price_per_tb = 5 

# Query Q1
query_q1 = """
SELECT ROUND((SUM(area_water_meters) / (SUM(area_land_meters) + SUM(area_water_meters))) * 100, 2) AS water_percentage
FROM 
  bigquery-public-data.geo_us_boundaries.counties
WHERE 
  county_name = 'Dane' AND state_fips_code = '55'
"""
# Query Q2
query_q2 = """
SELECT 
  s.state,
  COUNT(c.county_fips_code) AS num_counties
FROM 
  bigquery-public-data.geo_us_boundaries.counties c
JOIN 
  bigquery-public-data.geo_us_boundaries.states s
ON 
  c.state_fips_code = s.state_fips_code
GROUP BY 
  s.state
ORDER BY 
  num_counties DESC
LIMIT 5
"""

job_config = bigquery.QueryJobConfig(use_query_cache=False)

query_job_q1 = client.query(query_q1, job_config=job_config)
result_q1 = query_job_q1.result()  

query_job_q2 = client.query(query_q2, job_config=job_config)
result_q2 = query_job_q2.result()  

def calculate_cost(bytes_billed):
    price_per_tb = 6.25  
    return (bytes_billed / (1024**4)) * price_per_tb  

cost_q1 = calculate_cost(query_job_q1.total_bytes_billed)
cost_q2 = calculate_cost(query_job_q2.total_bytes_billed)

query_costs = {'q1': float(cost_q1), 'q2': float(cost_q2)}
query_costs

{'q1': 5.9604644775390625e-05, 'q2': 0.00011920928955078125}

In [6]:
#Q4
from google.cloud import bigquery

client = bigquery.Client()

dataset_id = f"{client.project}.p8"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)  

bucket_name = 'gcp_bucket_544'
file_name = 'hdma-wi-2021-split.parquet'
uri = f"gs://{bucket_name}/{file_name}"

table_id = f"{dataset_id}.hdma"
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite the table
)
load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  

load_job.result()

datasets = [ds.dataset_id for ds in client.list_datasets()]
datasets


['p8']

In [7]:
#Q5
query = """
SELECT c.county_name as county_name, COUNT(*) as application_count
FROM `p8.hdma` as h 
JOIN `bigquery-public-data.geo_us_boundaries.counties` as c
ON h.county_code = c.geo_id
WHERE c.state_fips_code = '55' 
GROUP BY county_name
ORDER BY application_count DESC
LIMIT 10
"""

result = bq.query(query).to_dataframe()

top_counties = dict(zip(result['county_name'], result['application_count']))
top_counties


{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [8]:
#Q6

#url = "https://docs.google.com/spreadsheets/d/1FfalqAWdzz01D1zIvBxsDWLW05-lvANWjjAj2vI4A04/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = ["https://docs.google.com/spreadsheets/d/1FfalqAWdzz01D1zIvBxsDWLW05-lvANWjjAj2vI4A04/"]
external_config.options.skip_leading_rows = 1  # Skip the header row
external_config.autodetect = True  # Auto-detect the schema

dataset_id = f"{client.project}.p8"
table_id = f"{dataset_id}.applications"
table = bigquery.Table(table_id)
table.external_data_configuration = external_config


client.create_table(table, exists_ok=True)

chosen_income = 100000 

query = f"""
SELECT 
    COUNT(*) AS num_applications
FROM 
    `{client.project}.p8.applications`
WHERE 
    income = {chosen_income}
"""

query_job = client.query(query)
results = query_job.result()
num_applications = 0

for row in results:
    num_applications = row.num_applications

num_applications


32

In [9]:
#Q7
query = f"""
WITH county_data AS (
  SELECT 
      county_name, 
      county_geom
  FROM 
      `bigquery-public-data.geo_us_boundaries.counties`
  WHERE 
      state_fips_code = '55'
)

SELECT 
    c.county_name,
    COUNT(*) AS application_count
FROM 
    county_data c
JOIN 
    `{client.project}.p8.applications` a
ON 
    ST_CONTAINS(c.county_geom, ST_GEOGPOINT(a.longitude, a.latitude))
GROUP BY 
    c.county_name
ORDER BY 
    application_count DESC
"""

query_job = client.query(query)
results = query_job.result()

county_applications = {row.county_name: row.application_count for row in results}
county_applications


{'Dane': 52,
 'Door': 12,
 'Milwaukee': 8,
 'Walworth': 8,
 'Brown': 4,
 'Jefferson': 4,
 'Sheboygan': 3,
 'Oneida': 3,
 'Winnebago': 3,
 'Marinette': 3,
 'Outagamie': 2,
 'Columbia': 2,
 'Manitowoc': 2,
 'Monroe': 2,
 'Douglas': 2,
 'Bayfield': 2,
 'Iron': 1,
 'Clark': 1,
 'Florence': 1,
 'Kewaunee': 1,
 'Fond du Lac': 1,
 'Juneau': 1,
 'Green Lake': 1,
 'Portage': 1,
 'Sauk': 1,
 'Marathon': 1,
 'Waupaca': 1,
 'Ashland': 1,
 'Barron': 1}

In [10]:
# Setup Part 4
from google.cloud import bigquery

client = bigquery.Client()

dataset_id = f"{client.project}.p8"
model_id = f"{dataset_id}.loan_prediction_model"

query_create_model = f"""
CREATE OR REPLACE MODEL `{model_id}`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['loan_amount']) AS
SELECT
  income,
  loan_term,
  loan_amount
FROM
  `{dataset_id}.hdma`
WHERE
  dataset = 'train'
"""

client.query(query_create_model).result()
print("Model training initiated.")


Model training initiated.


In [11]:
#Q8

query_evaluate_model = f"""
SELECT
  mean_absolute_error
FROM
  ML.EVALUATE(MODEL `{model_id}`, (
    SELECT
      income,
      loan_term,
      loan_amount
    FROM
      `{dataset_id}.hdma`
    WHERE
      dataset = 'test'
  ))
"""

mae_result = client.query(query_evaluate_model).to_dataframe()
mae_result['mean_absolute_error'].iloc[0]


77294.68408464032

In [12]:
#Q9
from google.cloud import bigquery
client = bigquery.Client()

dataset_id = f"{client.project}.p8"
model_id = f"{dataset_id}.loan_prediction_model"

query_model_weights = f"""
SELECT
  processed_input,
  weight
FROM
  ML.WEIGHTS(MODEL `{model_id}`)
"""


weights_result = client.query(query_model_weights).to_dataframe()
loan_term_coefficient = weights_result[weights_result['processed_input'] == 'loan_term']['weight'].iloc[0]

loan_term_coefficient


381.80803658694055

In [13]:
#q10
ratio_query = f"""
WITH predictions AS (
  SELECT 
    loan_amount AS actual_loan_amount, 
    predicted_loan_amount  
  FROM ML.PREDICT(MODEL `{model_id}`, (
      SELECT loan_amount, income, 360 as loan_term 
      FROM `{client.project}.p8.applications`
  ))
)
SELECT
    AVG(CASE WHEN actual_loan_amount > predicted_loan_amount THEN 1 ELSE 0 END) AS ratio_above_predicted
FROM
    predictions
"""

# Run the query to calculate the ratio
ratio_job = client.query(ratio_query)
ratio_result = ratio_job.to_dataframe()

ratio_result.iloc[0].mean()


0.5950000000000002